In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

# 데이터 로딩

In [ ]:
trainset_raw = torchvision.datasets.MNIST(root='./data', train=True, download=True)
testset_raw = torchvision.datasets.MNIST(root='./data', train=False, download=True)

In [ ]:
for row in trainset_raw.data[0].numpy():
    print(" ".join(f"{v:3}" for v in row))

In [ ]:
# numpy → tensor 변환 & 정규화
def preprocess_mnist(dataset):
    images = dataset.data.float() / 255.0  # [0,255] → [0,1]
    mean, std = images.mean() , images.std()
    images = (images - mean) / std         # Normalize
    labels = dataset.targets
    return images, labels

In [ ]:
train_images, train_labels = preprocess_mnist(trainset_raw)
test_images, test_labels = preprocess_mnist(testset_raw)

In [ ]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

In [ ]:
# Dataset 객체로 묶기
full_dataset = torch.utils.data.TensorDataset(train_images, train_labels)

In [ ]:
# train/val split (Dataset을 그대로 분할)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델 생성

In [ ]:
# !pip install torchinfo torchview

In [ ]:
from torchinfo import summary
from torchview import draw_graph

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.flat = nn.Flatten(start_dim=1)   # (N, 1, 28, 28) -> (N, 784)
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flat(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
model = DNN().to(device)  # 모델을 GPU로 이동
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
summary(model, input_size=(1, 28, 28))

In [ ]:
draw_graph(model, input_size=(1, 28, 28)).visual_graph

# 학습

In [ ]:
# 로그 저장용 리스트
train_losses, train_accuracies, valid_losses, valid_accuracies = [], [], [], []

In [ ]:
for epoch in range(30):                       # 총 40번 데이터셋을 반복
    model.train()                             # 학습 모드(드롭아웃/배치정규화 등 학습 동작 켜짐)
    running_loss = 0                          # 에폭 동안의 학습 손실 누적 변수
    train_correct, train_total = 0, 0         # (추가) 에폭 단위 Train Acc 계산용

    # ---- 미니배치 학습 루프 ----
    for inputs, labels in trainloader:        # DataLoader에서 (입력, 정답) 배치를 꺼냄
        inputs, labels = inputs.to(device), labels.to(device)  # 배치 텐서를 모델과 같은 장치로 이동
        optimizer.zero_grad()                 # 직전 step의 누적 gradient를 0으로 초기화
        outputs = model(inputs)               # 순전파: 모델이 logits(미규격화 점수) 출력
        loss = loss_fn(outputs, labels)       # 손실 계산(CrossEntropyLoss: softmax+NLL 통합)
        loss.backward()                       # 역전파: 각 파라미터의 gradient 계산
        optimizer.step()                      # 가중치 갱신(gradient를 이용해 한 스텝 업데이트)
        running_loss += loss.item()           # 현재 배치의 손실 값을 파이썬 float로 누적

        # (추가) 배치 예측으로 Train Acc 누적
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(trainloader))  # 에폭 평균 학습 손실 기록 (배치 개수로 나눔)
    train_accuracies.append(100 * train_correct / train_total)     # 학습 정확도(%) 기록

    # ----- Validation -----
    model.eval()                              # 평가 모드(드롭아웃/BN 등 평가 동작)
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():                     # 평가 시에는 gradient 계산 비활성화(메모리/속도 이점)
        for inputs, labels in valloader:      # 검증 데이터 배치 반복
            inputs, labels = inputs.to(device), labels.to(device)  # 장치 정렬
            outputs = model(inputs)           # 순전파만 수행
            loss = loss_fn(outputs, labels)   # 검증 배치 손실
            val_loss += loss.item()           # 손실 누적
            _, predicted = torch.max(outputs, 1)     # 각 샘플의 최고 점수 클래스 인덱스
            total += labels.size(0)                  # 총 샘플 수 누적
            correct += (predicted == labels).sum().item()  # 맞춘 개수 누적

    valid_losses.append(val_loss / len(valloader))     # 에폭 평균 검증 손실 기록
    valid_accuracies.append(100 * correct / total)     # 검증 정확도(%) 기록

    # 진행 상황 출력(총 30에 맞추어 표기)
    print(f"Epoch {epoch+1}/30 : TRAIN[Loss: {train_losses[-1]:.4f}, Acc: {train_accuracies[-1]:.2f}%], VALID[Loss: {valid_losses[-1]:.4f}, Acc: {valid_accuracies[-1]:.2f}%]")

# 학습결과 확인

In [ ]:
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(valid_losses, label="Valid Loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(valid_accuracies, label="Valid Accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()